In [19]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
import numpy as np
import matplotlib.pyplot as plt

In [20]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [21]:
# 입력 이미지 차원
img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# 정규화
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# 클래스 벡터를 이진 클래스 행렬로 변환
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# CNN 모델 생성
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [22]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=12,
          verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/12
469/469 [==============================] - 34s 72ms/step - loss: 2.2846 - accuracy: 0.1434 - val_loss: 2.2562 - val_accuracy: 0.2347
Epoch 2/12
469/469 [==============================] - 34s 73ms/step - loss: 2.2409 - accuracy: 0.2388 - val_loss: 2.2041 - val_accuracy: 0.3689
Epoch 3/12
469/469 [==============================] - 35s 75ms/step - loss: 2.1863 - accuracy: 0.3247 - val_loss: 2.1359 - val_accuracy: 0.4965
Epoch 4/12
469/469 [==============================] - 35s 74ms/step - loss: 2.1136 - accuracy: 0.4090 - val_loss: 2.0437 - val_accuracy: 0.6217
Epoch 5/12
469/469 [==============================] - 35s 74ms/step - loss: 2.0152 - accuracy: 0.4860 - val_loss: 1.9170 - val_accuracy: 0.7071
Epoch 6/12
469/469 [==============================] - 35s 75ms/step - loss: 1.8842 - accuracy: 0.5447 - val_loss: 1.7477 - val_accuracy: 0.7526
Epoch 7/12
469/469 [==============================] - 35s 74ms/step - loss: 1.7205 - accuracy: 0.5913 - val_loss: 1.5424 - val_accuracy:

In [23]:
# FGSM 공격 함수
def fgsm(model, x, eps):
    x_adv = tf.Variable(x, dtype=tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(x_adv)
        y_pred = model(x_adv)
        loss = tf.keras.losses.categorical_crossentropy(y_test, y_pred)
    grad = tape.gradient(loss, x_adv)
    x_adv = x_adv + eps * tf.sign(grad)
    x_adv = tf.clip_by_value(x_adv, 0, 1)
    return x_adv

In [24]:
# 적대적 공격으로 만든 이미지 생성
eps = 0.3
x_test_adv = fgsm(model, x_test, eps)

# 모델 성능 평가
score = model.evaluate(x_test_adv, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 3.081878423690796
Test accuracy: 0.033799998462200165
